In [1]:
import gym
import random, math
import numpy as np
import arcade
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from PIL import Image

        
from LightEnvCopy import LightEnv

import gym.spaces
from gym.spaces import Discrete, Box

from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog

from collections import namedtuple

# Do the math to figure out our screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Game 1: Let There Be Light!"

# COnvenient data structure to hold information about actions
Action = namedtuple('Action', 'name index delta_i delta_j')

up = Action('up', 0, -1, 0)    
down = Action('down', 1, 1, 0)    
left = Action('left', 2, 0, -1)    
right = Action('right', 3, 0, 1)    

index_to_actions = {}
for action in [up, down, left, right]:
    index_to_actions[action.index] = action
# print(index_to_actions[0].name)
str_to_actions = {}
for action in [up, down, left, right]:
    str_to_actions[action.name] = action
#TF End - Adding in actions for action conversion


class LightEnvWrapper(gym.Env, LightEnv):
    """Class that wraps the Lights Environment to make it 
    compatible with RLLib."""

    metadata = {"render.modes": ["rgb_array", "state_pixels"]}
    
    def __init__(self, config: EnvContext):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        self.torch_collected = False
        self.torch_collected_count = []
        self.mygame = LightEnv
        self.steps_taken = 0
        #The action space is a choice of 4 actions: U/D/L/R.
        self.action_space = Discrete(4)
        
        #The observation space is a fixed image of the current game screen
        self.observation_space = Box(low=0, high=255, shape=(84,84, 4), dtype=np.uint8)
        
    def reset(self):
        print("resetting in wrapper")
        
        if self.torch_collected == 1:
            print("Torch was collected this episode!")
        else:
            print("Torch was not collected this episode...")
        self.torch_collected_count.append(self.torch_collected)
        print(self.torch_collected_count)

        self.render(self)
        #Resets the state of the environment for a new episode and an initial observation.
        obs_mygame = self.mygame.reset(self)
        
        #Open up the resetted image to verify working correctly.
        obs_mygame.show()
        
        self.mygame.on_draw(self)
        #Convert observation to 84x84 resolution and np array for rllib.
        obs = self.convert_observations(obs_mygame)
        
        self.steps_taken = 0
        return obs

    def step(self, action):
        self.steps_taken += 1
        
        #Making sure an action is chosen, either: 0, 1, 2, 3.
        assert action in [0, 1, 2, 3] #0-up,1-down,2-left,3-right.
        
        #Convert the numeric action to a keyword: up, down, left, right.
        actions_myenv = index_to_actions[action].name #returns a word, one of: up/down/left/right
#         print(f"action taken: {actions_myenv}")
        
        #Update the window with on_update()
        self.render(self)
#         print("env rendered")
        #Compute observation extracted from the window (800x600), with reward and done flag.
        obs, reward, done, torch_collected, fps_check = self.mygame.step(self,actions_myenv)
        if torch_collected == True:
            self.torch_collected = 1
        else:
            self.torch_collected = 0
                    
        if self.steps_taken % 100 == 0: #33 steps roughly equates to 1 second in game time
            print(f"total score is {self.score} at time: {self.mygame.time_taken_reported(self)}")
            print(f"FPS is currently: {fps_check}")
        #Convert observation to 84x84 resolution and np array for rllib.
        obs_mygame = self.convert_observations(obs)
        
        #If the reward has been obtained, reset the environment and start again
        if done == True:
            print(f"done is {done}, resetting environment in wrapper.")
            print(f"steps taken: {self.steps_taken}")
            obs.show()
            self.reset()
        
        return obs_mygame, reward, done, {}

    def seed(self, seed=None):
        random.seed(seed)

    def convert_observations(self, obs_mygame): #resizing and converting to array for rllib processing
        # We normalize and concatenate observations
        obs = obs_mygame
        obs_resized = obs.resize((84,84))
        obsarray = np.array(obs_resized)
        return obsarray
    
    def render(self, mode='state_pixels'):
#         self.mygame.update(self)
        self.mygame.on_draw(self)
        test = self.mygame.time_taken_reported(self)
        

C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


### Now run the rllib script to train the agent

### Manual Grid Search

#### lr

In [2]:
import gym
import ray.rllib.agents.ppo.ppo as ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
import os


config = PPOConfig()


from ray.rllib.algorithms.ppo import PPOConfig
RAY_DISABLE_MEMORY_MONITOR = 1
config = PPOConfig().training(gamma=0.99, lr=0.001, kl_coeff=0.2, entropy_coeff=1,
#                               entropy_coeff_schedule=[[0,1],[1000,0]],
                             sgd_minibatch_size=128, num_sgd_iter=60)\
            .resources(num_gpus=0)\
            .rollouts(num_envs_per_worker=1, num_rollout_workers=2, recreate_failed_workers=True, horizon=30000)
config.normalize_actions=False
config.env=LightEnvWrapper
config.clip_actions=False
print(config.to_dict())
# Build a Algorithm object from the config
trainer = ppo.PPOTrainer(config=config)

avg_rewards = []
num_iterations = []
for episode in range(100):
    print("Starting episode ", episode)
    # Perform one iteration of training the policy with PPO
    result = trainer.train()
    #print(pretty_print(result))
    print("episode reward mean: ", result['episode_reward_mean'])
    avg_rewards.append(result['episode_reward_mean'])
    num_iterations.append(episode)
    if episode % 10 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)
    print("End of episode ", episode)



    

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': <class '__main__.LightEnvWrapper'>, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': False, 'clip_actions': False, 'disable_env_checking': False, 'num_workers': 2, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollecto

C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\_private\ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-09-27 21:46:37,332	INFO worker.py:1518 -- Started a local Ray instance.
(pid=21952) Windows fatal exception: code 0xc0000139
(pid=21952) 
(pid=5152) Windows fatal exception: code 0xc0000139
(pid=5152) 
(pid=21952) C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=21952)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=5152) C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\pyth

(RolloutWorker pid=21952) resetting in wrapper
(RolloutWorker pid=21952) Torch was not collected this episode...
(RolloutWorker pid=21952) [False]
(RolloutWorker pid=5152) resetting in wrapper
(RolloutWorker pid=5152) Torch was not collected this episode...
(RolloutWorker pid=5152) [False]
(RolloutWorker pid=21952) resetting
(RolloutWorker pid=5152) resetting


2022-09-27 21:47:01,755	INFO trainable.py:160 -- Trainable.setup took 29.034 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-27 21:47:01,758	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Starting episode  0
(RolloutWorker pid=21952) resetting in wrapper
(RolloutWorker pid=21952) Torch was not collected this episode...
(RolloutWorker pid=21952) [False, 0]
(RolloutWorker pid=21952) resetting
(RolloutWorker pid=5152) resetting in wrapper
(RolloutWorker pid=5152) Torch was not collected this episode...
(RolloutWorker pid=5152) [False, 0]
(RolloutWorker pid=5152) resetting
(RolloutWorker pid=5152) total score is -4 at time: 2
(RolloutWorker pid=5152) FPS is currently: 88
(RolloutWorker pid=21952) total score is -4 at time: 2
(RolloutWorker pid=21952) FPS is currently: 80
(RolloutWorker pid=5152) total score is -5 at time: 3
(RolloutWorker pid=5152) FPS is currently: 93
(RolloutWorker pid=21952) total score is -5 at time: 3
(RolloutWorker pid=21952) FPS is currently: 91
(RolloutWorker pid=5152) total score is -7 at time: 5
(RolloutWorker pid=5152) FPS is currently: 82
(RolloutWorker pid=21952) total score is -7 at time: 5
(RolloutWorker pid=21952) FPS is currently: 80
(Rollo

...

(RolloutWorker pid=27112) total score is -500 at time: 460
(RolloutWorker pid=27112) FPS is currently: 86
(RolloutWorker pid=940) total score is -37 at time: 27
(RolloutWorker pid=940) FPS is currently: 88
(RolloutWorker pid=27112) total score is -502 at time: 462
(RolloutWorker pid=27112) FPS is currently: 101
(RolloutWorker pid=940) total score is -39 at time: 28
(RolloutWorker pid=940) FPS is currently: 90
(RolloutWorker pid=27112) total score is -503 at time: 463
(RolloutWorker pid=27112) FPS is currently: 91
(RolloutWorker pid=940) total score is -42 at time: 30
(RolloutWorker pid=940) FPS is currently: 92
(RolloutWorker pid=27112) total score is -505 at time: 465
(RolloutWorker pid=27112) FPS is currently: 88
(RolloutWorker pid=940) total score is -44 at time: 32
(RolloutWorker pid=940) FPS is currently: 78
(RolloutWorker pid=27112) total score is -507 at time: 467
(RolloutWorker pid=27112) FPS is currently: 90
(RolloutWorker pid=940) total score is -46 at time: 33
(RolloutWorker

(RolloutWorker pid=27112) total score is -14 at time: 12
(RolloutWorker pid=27112) FPS is currently: 106
(RolloutWorker pid=940) total score is -95 at time: 78
(RolloutWorker pid=940) FPS is currently: 112
(RolloutWorker pid=27112) total score is -15 at time: 13
(RolloutWorker pid=27112) FPS is currently: 102
(RolloutWorker pid=940) total score is -97 at time: 80
(RolloutWorker pid=940) FPS is currently: 110
(RolloutWorker pid=27112) total score is -17 at time: 15
(RolloutWorker pid=27112) FPS is currently: 85
(RolloutWorker pid=940) total score is -99 at time: 82
(RolloutWorker pid=940) FPS is currently: 81
(RolloutWorker pid=27112) total score is -19 at time: 17
(RolloutWorker pid=27112) FPS is currently: 92
(RolloutWorker pid=940) total score is -101 at time: 83
(RolloutWorker pid=940) FPS is currently: 95
(RolloutWorker pid=27112) total score is -20 at time: 18
(RolloutWorker pid=27112) FPS is currently: 93
(RolloutWorker pid=940) total score is -103 at time: 85
(RolloutWorker pid=

(RolloutWorker pid=940) total score is -143 at time: 122
(RolloutWorker pid=940) FPS is currently: 91
(RolloutWorker pid=27112) total score is -27 at time: 20
(RolloutWorker pid=27112) FPS is currently: 99
(RolloutWorker pid=940) total score is -145 at time: 123
(RolloutWorker pid=940) FPS is currently: 81
(RolloutWorker pid=27112) total score is -30 at time: 22
(RolloutWorker pid=27112) FPS is currently: 85
(RolloutWorker pid=940) total score is -147 at time: 125
(RolloutWorker pid=940) FPS is currently: 95
(RolloutWorker pid=27112) total score is -31 at time: 23
(RolloutWorker pid=27112) FPS is currently: 87
(RolloutWorker pid=940) total score is -150 at time: 127
(RolloutWorker pid=940) FPS is currently: 83
(RolloutWorker pid=27112) total score is -33 at time: 25
(RolloutWorker pid=27112) FPS is currently: 92
(RolloutWorker pid=940) total score is -151 at time: 128
(RolloutWorker pid=940) FPS is currently: 97
(RolloutWorker pid=27112) total score is -35 at time: 27
(RolloutWorker pi

C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\rllib\policy\dynamic_tf_policy_v2.py:517: DeprecationWarning: invalid escape sequence \d
  mo = re.match("state_in_(\d+)", view_col)


KeyboardInterrupt: 